# नमूना ०१: OpenAI SDK मार्फत छिटो कुराकानी

यो नोटबुकले Microsoft Foundry Local सँग OpenAI SDK प्रयोग गरेर छिटो कुराकानी गर्ने तरिका देखाउँछ।

## अवलोकन

यो उदाहरणले देखाउँछ:
- Foundry Local सँग OpenAI Python SDK प्रयोग गर्ने
- Azure OpenAI र स्थानीय Foundry कन्फिगरेसनहरू दुवैलाई व्यवस्थापन गर्ने
- उचित त्रुटि व्यवस्थापन र फलब्याक रणनीतिहरू कार्यान्वयन गर्ने
- सेवा व्यवस्थापनको लागि FoundryLocalManager प्रयोग गर्ने


## आवश्यकताहरू

निश्चित गर्नुहोस् कि तपाईंले:
1. Foundry Local स्थापना र चलाइराख्नुभएको छ
2. एउटा मोडेल लोड गरिएको छ (जस्तै, `foundry model run phi-4-mini`)
3. आवश्यक Python प्याकेजहरू स्थापना गर्नुभएको छ

### निर्भरता स्थापना


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## आवश्यक पुस्तकालयहरू आयात गर्नुहोस्


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## कन्फिगरेसन

आफ्नो कन्फिगरेसन सेटअप गर्नुहोस्। तपाईं निम्नमध्ये कुनै एक प्रयोग गर्न सक्नुहुन्छ:
1. **Azure OpenAI** (क्लाउड-आधारित)
2. **Foundry Local** (स्थानीय इनफरेन्स)

### विकल्प १: Azure OpenAI कन्फिगरेसन


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### विकल्प २: फाउन्ड्री स्थानीय कन्फिगरेसन


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## क्लाइन्ट आरम्भ

तपाईंको कन्फिगरेसनको आधारमा OpenAI क्लाइन्ट सिर्जना गर्नुहोस्:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## साधारण च्याट उदाहरण

साधारण च्याट अन्तरक्रिया परीक्षण गरौं:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## अन्तरक्रियात्मक च्याट सत्र

मोडेलले कसरी प्रतिक्रिया दिन्छ भनेर हेर्न विभिन्न प्रकारका प्रम्प्टहरू प्रयास गर्नुहोस्:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## उन्नत प्रयोग: स्ट्रिमिङ प्रतिक्रियाहरू

लामो प्रतिक्रियाहरूको लागि, स्ट्रिमिङले राम्रो प्रयोगकर्ता अनुभव प्रदान गर्न सक्छ:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## त्रुटि व्यवस्थापन र निदान

सेवा स्थिति र उपलब्ध मोडेलहरू जाँच गरौं:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## कस्टम प्रम्प्ट परीक्षण

तपाईंको आफ्नै प्रम्प्टहरू परीक्षण गर्न तलको सेल प्रयोग गर्नुहोस्:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## सारांश

यस नोटबुकले निम्न विषयहरू प्रदर्शन गर्यो:

1. **✅ क्लाइन्ट सेटअप**: Foundry Local सँग OpenAI SDK कसरी कन्फिगर गर्ने
2. **✅ आधारभूत च्याट**: सरल अनुरोध-प्रतिक्रिया अन्तरक्रिया
3. **✅ स्ट्रिमिङ**: वास्तविक-समय प्रतिक्रिया उत्पादन
4. **✅ त्रुटि व्यवस्थापन**: बलियो त्रुटि व्यवस्थापन र डायग्नोस्टिक्स
5. **✅ सेवा स्वास्थ्य**: मोडेल उपलब्धता र सेवा स्थिति जाँच गर्ने

### आगामी कदमहरू

- **नमूना 02**: Azure OpenAI समर्थनसहितको उन्नत SDK एकीकरण
- **नमूना 04**: Chainlit च्याट एप्लिकेसन निर्माण
- **नमूना 05**: बहु-एजेन्ट समन्वय प्रणालीहरू
- **नमूना 06**: बौद्धिक मोडेल रुटिङ

### Foundry Local का प्रमुख फाइदाहरू

- 🔒 **गोपनीयता**: डेटा कहिल्यै तपाईंको उपकरणबाट बाहिर जाँदैन
- ⚡ **गति**: न्यून विलम्बता स्थानीय अनुमान
- 💰 **खर्च**: कुनै API प्रयोग लागत छैन
- 🔌 **अफलाइन**: इन्टरनेट जडान बिना काम गर्दछ
- 🛠️ **अनुकूलता**: OpenAI-संग मिल्दो API
